### Pipeline IPTU — Leitura de Excel, Normalização e Exportação para Parquet (Staging)

Este notebook realiza o processamento em lote de arquivos Excel localizados no diretório *Downloads*,
filtrando abas com prefixos de mês/ano, aplicando padronizações de campos (ex.: SQL/IPTU), tipagens
(numéricos, monetários e datas), enriquecimento com colunas derivadas e persistindo a saída em formato
Parquet particionado por **ano** e **mês**.

### 1) Importações

Bibliotecas utilizadas:
- **pandas**: leitura/manipulação de dados tabulares.
- **pathlib**: manipulação segura de caminhos (independente de SO).
- **os**: suporte para variáveis de ambiente e listagem de arquivos.

In [3]:
!pip install pandas openpyxl pyarrow -q

In [4]:
import os
import pandas as pd
from pathlib import Path

### 2) Funções utilitárias

Nesta seção estão as funções responsáveis por:
- **Padronizar o identificador SQL/IPTU** para o formato `000.000.0000-0` (11 dígitos).
- **Converter moeda no padrão BRL** (`1.234,56`) para `float` (`1234.56`).

In [5]:
def formatar_sql_iptu(col: pd.Series) -> pd.Series:
    """
    Padroniza o identificador SQL/IPTU para o formato: 000.000.0000-0 (11 dígitos).

    Regras aplicadas:
    - Converte para numérico (nullable) com coerção: valores inválidos viram NA.
    - Remove caracteres não numéricos (pontuação, espaços, etc.).
    - Mantém apenas os últimos 11 dígitos (quando houver ruído/prefixos).
    - Completa com zeros à esquerda até 11 dígitos.
    - Aplica máscara final no padrão esperado.
    """
    s = pd.to_numeric(col, errors="coerce").astype("Int64")
    s = s.astype(str).replace("<NA>", pd.NA)
    s = s.str.replace(r"\D", "", regex=True)
    s = s.str[-11:]
    s = s.str.zfill(11)
    s = s.str.replace(r"^(\d{3})(\d{3})(\d{4})(\d)$", r"\1.\2.\3-\4", regex=True)
    return s


def moeda_brl_para_float(series: pd.Series, errors: str = "coerce") -> pd.Series:
    """
    Converte valores monetários no padrão brasileiro (ex.: '1.234,56') para float (1234.56).

    Observações:
    - Remove separadores de milhar '.' e substitui separador decimal ',' por '.'.
    - O parâmetro `errors` controla a política de erro do pandas:
      - 'coerce': valores inválidos viram NaN
      - 'raise' : interrompe o processamento ao encontrar valor inválido
    """
    s = series.astype(str)
    s = s.str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors=errors)


### 3) Configuração de diretórios

Definições principais:
- **downloads_path**: origem dos arquivos Excel (padrão `~/downloads`).
- **staging_dir**: diretório local de apoio (opcional).

**Observação:** a saída final em Parquet será escrita em um caminho absoluto no padrão:
`~/ĩtbi-ml/staging/bronze/ano=<YY>/mes=<MMM>/`

In [6]:
print("Diretório de trabalho atual (CWD):", os.getcwd())

bronze_dir = Path("../data/bronze/itbi")
staging_dir = Path("../data/staging/itbi")

# Garante que os diretórios existam
bronze_dir.mkdir(parents=True, exist_ok=True)
staging_dir.mkdir(parents=True, exist_ok=True)

print("Diretório staging:", staging_dir.resolve())
print("Diretório bronze:", bronze_dir.resolve())

Diretório de trabalho atual (CWD): /home/ozolsvoz/projects/itbi-ml/notebooks
Diretório staging: /home/ozolsvoz/projects/itbi-ml/data/staging/itbi
Diretório bronze: /home/ozolsvoz/projects/itbi-ml/data/bronze/itbi


### 4) Filtro de abas válidas

Apenas abas cujo nome inicia com os prefixos abaixo serão processadas.
Isso evita ingestão de abas auxiliares, sumários ou conteúdos fora do escopo.

In [7]:
prefixos_abas = (
    "JAN-20","FEV-20","MAR-20","ABR-20","MAI-20","JUN-20",
    "JUL-20","AGO-20","SET-20","OUT-20","NOV-20","DEZ-20"
)


### 5) Descoberta de arquivos Excel

Lista todos os arquivos `.xlsx` e `.xls` no diretório de Downloads para processamento em lote.

In [8]:
excel_files = [p for p in staging_dir.iterdir() if p.is_file() and p.suffix.lower() in (".xlsx", ".xls")]
print("Quantidade de arquivos Excel encontrados no staging:", len(excel_files))
excel_files[:10]

Quantidade de arquivos Excel encontrados no staging: 20


[PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2008.xlsx'),
 PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2006.xlsx'),
 PosixPath('../data/staging/itbi/GUIAS_DE_ITBI_PAGAS_12-2022.xlsx'),
 PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2007.xlsx'),
 PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2009.xlsx'),
 PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2011.xlsx'),
 PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2010.xlsx'),
 PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2012.xlsx'),
 PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2013.xlsx'),
 PosixPath('../data/staging/itbi/guias_de_itbi_pagas_2014.xlsx')]

### 6) Dicionário de padronização de colunas (Português em snake_case)

Este mapeamento converte os nomes originais (como vêm no Excel) para um padrão consistente:
- **português**
- **snake_case**

Observação: campos derivados também seguem o mesmo padrão.

In [10]:
mapeamento_colunas = {
    "N° do Cadastro (SQL)": "cadastro_sql",
    "Nome do Logradouro": "nome_logradouro",
    "Número": "numero_imovel",
    "Complemento": "complemento_imovel",
    "Natureza de Transação": "natureza_transacao",
    "Valor de Transação (declarado pelo contribuinte)": "valor_transacao_declarado",
    "Data de Transação": "data_transacao",
    "Valor Venal de Referência": "valor_venal_referencia",
    "Proporção Transmitida (%)": "proporcao_transmitida_percent",
    "Valor Venal de Referência (proporcional)": "valor_venal_referencia_proporcional",
    "Situação do SQL": "situacao_sql",
    "Área do Terreno (m2)": "area_terreno_m2",
    "Testada (m)": "testada_m",
    "Fração Ideal": "fracao_ideal",
    "Área Construída (m2)": "area_construida_m2",
    "Uso (IPTU)": "uso_iptu",
    "Descrição do uso (IPTU)": "descricao_uso_iptu",
    "Padrão (IPTU)": "padrao_iptu",
    "Descrição do padrão (IPTU)": "descricao_padrao_iptu",
    "ACC (IPTU)": "ano_conclusao_construcao_iptu",
}

### 7) Processamento e exportação

Para cada arquivo Excel encontrado:
1. Abre o arquivo e percorre suas abas.
2. Processa apenas as abas com prefixo válido (ex.: `JAN-20`).
3. Lê o conteúdo **como string** para controlar tipagens manualmente.
4. Aplica:
   - Normalização do `N° do Cadastro (SQL)`.
   - Conversões monetárias, percentuais, medidas e datas.
   - Enriquecimento: `Setor Fiscal`, `Quadra`, `Mês`, `Ano`.
5. Escreve em Parquet particionado por ano/mês.

**Importante:** alguns campos usam `errors="raise"` por requisito de qualidade:
caso apareça valor inválido, o processamento deve falhar para evitar dados inconsistentes.

In [11]:
for caminho_arquivo in excel_files:
    print(f"\n=== Processando arquivo: {caminho_arquivo.name} ===")

    xls = pd.ExcelFile(caminho_arquivo)

    for aba in xls.sheet_names:
        if not aba.startswith(prefixos_abas):
            continue

        print(f"-> Aba selecionada: {aba}")

        # Leitura controlada: tudo como string para conversões explícitas
        df = pd.read_excel(xls, sheet_name=aba, dtype=str)

        # Renomeia colunas para português em snake_case
        df = df.rename(columns=mapeamento_colunas)

        # Padronização do identificador SQL/IPTU
        if "cadastro_sql" in df.columns:
            df["cadastro_sql"] = formatar_sql_iptu(df["cadastro_sql"])

        # Conversões monetárias e numéricas (padrão BRL)
        if "valor_transacao_declarado" in df.columns:
            df["valor_transacao_declarado"] = moeda_brl_para_float(
                df["valor_transacao_declarado"], errors="coerce"
            )

        if "valor_venal_referencia" in df.columns:
            df["valor_venal_referencia"] = moeda_brl_para_float(
                df["valor_venal_referencia"], errors="coerce"
            )

        # Requisito: falhar caso valor proporcional seja inválido
        if "valor_venal_referencia_proporcional" in df.columns:
            df["valor_venal_referencia_proporcional"] = moeda_brl_para_float(
                df["valor_venal_referencia_proporcional"], errors="raise"
            )

        # Percentual com separador decimal ','
        if "proporcao_transmitida_percent" in df.columns:
            df["proporcao_transmitida_percent"] = pd.to_numeric(
                df["proporcao_transmitida_percent"].astype(str).str.replace(",", ".", regex=False),
                errors="raise"
            )

        # Medidas físicas / áreas
        if "area_terreno_m2" in df.columns:
            df["area_terreno_m2"] = pd.to_numeric(df["area_terreno_m2"], errors="coerce")

        if "testada_m" in df.columns:
            df["testada_m"] = pd.to_numeric(df["testada_m"], errors="raise")

        if "area_construida_m2" in df.columns:
            df["area_construida_m2"] = pd.to_numeric(df["area_construida_m2"], errors="raise")

        if "ano_conclusao_construcao_iptu" in df.columns:
            df["ano_conclusao_construcao_iptu"] = pd.to_numeric(
                df["ano_conclusao_construcao_iptu"], errors="coerce", downcast="integer"
            )

        # Data da transação
        if "data_transacao" in df.columns:
            df["data_transacao"] = pd.to_datetime(df["data_transacao"], errors="coerce")

        # Enriquecimento: setor_fiscal e quadra a partir do cadastro_sql mascarado
        if "cadastro_sql" in df.columns:
            partes = df["cadastro_sql"].astype(str).str.split(".")
            df["setor_fiscal"] = partes.str[0]
            df["quadra"] = partes.str[1]
        else:
            df["setor_fiscal"] = pd.NA
            df["quadra"] = pd.NA

        # Partições: mes/ano a partir do nome da aba
        mes = aba.split("-")[0]
        ano = aba.split("-")[1]
        df["mes"] = mes
        df["ano"] = ano

        # Diretório de saída (Bronze) particionado
        output_dir = bronze_dir / f"ano={ano}" / f"mes={mes}"
        output_dir.mkdir(parents=True, exist_ok=True)

        # Mantém colunas originais e posiciona derivadas ao final
        colunas_derivadas = ["setor_fiscal", "quadra", "mes", "ano"]
        colunas_finais = [c for c in df.columns if c not in colunas_derivadas] + colunas_derivadas
        df = df[colunas_finais]

        # Escrita final
        out_file = output_dir / f"{aba}.parquet"
        df.to_parquet(out_file, index=False)

        print("OK ->", out_file)


=== Processando arquivo: guias_de_itbi_pagas_2008.xlsx ===
-> Aba selecionada: JAN-2008
OK -> ../data/bronze/itbi/ano=2008/mes=JAN/JAN-2008.parquet
-> Aba selecionada: FEV-2008
OK -> ../data/bronze/itbi/ano=2008/mes=FEV/FEV-2008.parquet
-> Aba selecionada: MAR-2008
OK -> ../data/bronze/itbi/ano=2008/mes=MAR/MAR-2008.parquet
-> Aba selecionada: ABR-2008
OK -> ../data/bronze/itbi/ano=2008/mes=ABR/ABR-2008.parquet
-> Aba selecionada: MAI-2008
OK -> ../data/bronze/itbi/ano=2008/mes=MAI/MAI-2008.parquet
-> Aba selecionada: JUN-2008
OK -> ../data/bronze/itbi/ano=2008/mes=JUN/JUN-2008.parquet
-> Aba selecionada: JUL-2008
OK -> ../data/bronze/itbi/ano=2008/mes=JUL/JUL-2008.parquet
-> Aba selecionada: AGO-2008
OK -> ../data/bronze/itbi/ano=2008/mes=AGO/AGO-2008.parquet
-> Aba selecionada: SET-2008
OK -> ../data/bronze/itbi/ano=2008/mes=SET/SET-2008.parquet
-> Aba selecionada: OUT-2008
OK -> ../data/bronze/itbi/ano=2008/mes=OUT/OUT-2008.parquet
-> Aba selecionada: NOV-2008
OK -> ../data/bronze/